# Preprocessing danych

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

#data = pd.read_csv('merged_data.csv')
data = pd.read_csv('fifth-run/feats_version_5_22_acc.csv')
data = data[~data["activity"].str.contains("jogging")]
data["activity"] = data["activity"].replace(["sitting", "lying"], "resting")

data = data.reset_index(drop=True)

result = data[data["name"].str.contains("kuba")]

result_train = data[data["name"].str.contains("kuba") == False]

result_train.drop('name', axis=1, inplace=True)
result.drop('name', axis=1, inplace=True)


X_train = result_train.iloc[:, :-1].values
labels_train = result_train.iloc[:, -1].values

X_test = result.iloc[:, :-1].values
labels_test = result.iloc[:, -1].values


encoder = LabelBinarizer()

labels_train1 = encoder.fit_transform(labels_train)

labels_test1 = encoder.transform(labels_test)

X_test, X_val, labels_test1, labels_val1 = train_test_split(X_test, labels_test1, test_size=0.5, random_state=42)

X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
X_val = np.array(X_val).reshape(X_val.shape[0], X_val.shape[1], 1)

print("Wielkość zbioru X Train: ", X_train.shape)
print("Wielkość zbioru X Test: ", X_test.shape)
print("Wielkość zbioru X Validation: ", X_val.shape)
print("Wielkość zbioru Y Train: ", labels_train1.shape)
print("Wielkość zbioru Y Test: ", labels_test1.shape)
print("Wielkość zbioru Y Validation: ", labels_val1.shape)

## Uczenie modelu

In [ ]:
cnn_model = tf.keras.models.Sequential()

cnn_model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.000000000001), input_shape = (X_train.shape[1],1)))
cnn_model.add(Conv1D(filters=128, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.00000000000001)))
cnn_model.add(MaxPool1D(pool_size=(2,), strides=2, padding='same'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Flatten())
cnn_model.add(Dense(units = 512, activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
cnn_model.add(Dense(units = 6, activation='softmax'))
cnn_model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

[print(i.shape, i.dtype) for i in cnn_model.inputs]
[print(o.shape, o.dtype) for o in cnn_model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in cnn_model.layers]

class_mapping = {
    0: "schodzenie",  
    1: "odpoczynek",
    2: "przysiady",
    3: "stanie",
    4: "wchodzenie",
    5: "chodzenie" }

listt =[]
for key in class_mapping.values():
    listt.append(key)

class BestAccuracyCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.best_accuracy = 0.0  

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get('val_accuracy')
        if val_accuracy is not None and val_accuracy > self.best_accuracy:
            self.best_accuracy = val_accuracy
    
    def on_train_end(self, logs=None):
        print(f"Best Validation Accuracy: {self.best_accuracy * 100:.2f}%")
        
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.01,  
    patience=60,  
    restore_best_weights=True
)
        
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='najlepszy_model_CNNprosty.h5',  # Filepath to save the best model
    monitor='val_accuracy',    
    save_best_only=True,       
)

cnn_model_history = cnn_model.fit(
    X_train, labels_train1, epochs=100, batch_size=60,
    validation_data=(X_test, labels_test1),
    callbacks=[early_stopping_callback, model_checkpoint],
    verbose = 1 # Include the model checkpoint callback
)

best_model = tf.keras.models.load_model('najlepszy_model_CNNprosty.h5')
y_pred = best_model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
labels_val_categorical = np.argmax(labels_val1, axis=1)
classification_rep = classification_report(labels_val_categorical, y_pred_classes, target_names=listt)
print("Classification Report from Best Model:\n", classification_rep)
confusion_mtx = confusion_matrix(labels_val_categorical, y_pred_classes)
accuracy = accuracy_score(labels_val_categorical, y_pred_classes)
print("Accuracy:", accuracy)
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=listt, yticklabels=listt)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix from Best Model')
plt.show()


